In [1]:
from Functionbase import *
from Backtest import *

In [ ]:
import os 
from dotenv import load_dotenv

load_dotenv()
api = sj.Shioaji(simulation=True)
api.login(
    os.getenv('API_KEY'),
    os.getenv('SECRET_KEY'),
    contracts_cb= lambda security_type: print(f"{repr(security_type)} fetch done"))

connection = sqlite3.connect('data.db')

In [6]:
ticks = {}
kbars = {}
volume_today = {}
position = {}

@api.quote.on_quote
def quote_callback(topic: str, quote: dict):

    code = topic.split('/')[-1]

    ts = pd.to_datetime(quote.get('Date')  + ' ' + quote.get('Time'))

    if ts.hour < 9:
        return
    
    close = quote.get('Close')[0]
    volume = quote.get('Volume')[0]
    vol_sum = quote.get('VolSum')[0]

    ticks[code].loc[ts] = [close, volume]
    volume_today[code] = vol_sum

In [16]:
import time
def tbot(code):
    import pandas as pd

    print(" ----- 啟動 Tbot  #{} -----".format(code))

    tw_calendar = get_calendar('XTAI')

    date = pd.to_datetime(datetime.datetime.now().date())
    prev_trading_date = tw_calendar.previous_close(date).date()

    if date not in tw_calendar.opens:
        print('今日非交易日')
        return
    
    ticks[code] = pd.concat([get_ticks(code, prev_trading_date, connection, api)[0], get_ticks(code, date, connection, api)[0]])
    ticks[code] = ticks[code][['close', 'volume']]
    kbars[code] = get_technical_indicator(ticks_to_kbars(ticks[code], interval='1Min'))
    kbars[code] = kbars[code][date:]
    volume_today[code] = kbars[code]['volume'].sum()

    api.quote.subscribe(api.Contracts.Stocks[code], quote_type = 'tick')

    while True:

        time.sleep(1)

        current_time = datetime.datetime.now()

        if current_time.second == 0:

            kbars[code] = get_technical_indicator(ticks_to_kbars(ticks[code], interval='1Min'))
            kbars[code] = kbars[code][date:]

            str_current_time = (current_time - datetime.timedelta(minutes=1)).strftime('%Y-%m-%d %H:%M:%S')
            
            try:
                
                current_price = kbars[code].loc[str_current_time]['close']
            
            except:
                continue

            position = position.get(code, 0)

            if (
                current_time >= current_time.replace(hour=9, minute=15, second=0) and
                current_time <=  current_time.replace(hour=9, minute=30, second=0) and 
                position == 0
            ):
                
                high_15m = kbars[code][:current_time.replace(hour=9, minute=14, second=0)]['high'].max()
                low_15m = kbars[code][:current_time.replace(hour=9, minute=14, second=0)]['low'].max()

                print(' [{}] 最高點:{}  |  最低點:{}  |  股價:{}  |  總成交量:{} '.format(str_current_time,
                                                                             high_15m,
                                                                             low_15m,
                                                                             current_price,
                                                                             volume_today[code]))
                
                if (
                    current_price > high_15m and
                    volume_today[code] > 2000
                ):
                    position[code] = 1

                    target_price = current_price * 1.03
                    stop_price = low_15m

                    print(' [{}] buy {} at {}'.format(str_current_time, code, current_price))

            
            elif(
                current_time >= current_time.replace(hour=9, minute=15, second=0) and
                current_time <=  current_time.replace(hour=13, minute=0, second=0) and 
                position != 0
            ):
                print('[{}] 停利:{} at {}'.format(current_time, code, current_price))
                break

            elif(
                current_time >= current_time.replace(hour=13, minute=0, second=0)
            ):
                break

            

    api.quote.unsubscribe(api.Contracts.Stocks[code], quote_type = 'tick')

    print('----- Close Tbot #{} -----'.format(code))
                


In [ ]:
tbot('2330')